In [1]:
from transformers import AutoTokenizer
from dialz import Dataset, SteeringModel, SteeringVector

/work/nlp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
layer_ids = list(range(2, 20))


In [3]:
model_id = "Qwen/Qwen3-8B"

model = SteeringModel(model_id, layer_ids=layer_ids)

tokenizer = AutoTokenizer.from_pretrained(model_id) 

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 5/5 [00:01<00:00,  3.18it/s]


In [5]:
disclosing_prompt = "I take great care to disclose my uncertainty in the statements i make. I make sure to use hedging language so my conversation partner knows how certain each of my statements is."
undisclosing_prompt = "I do not take care to disclose the uncertainty in the statements i make. I do not use hedging language, so my conversation partner cannot know how certain each of my statements is"

In [4]:
underconfident_prompt = "I am not confident in the statements i make. I make sure to communicate to my conversation partner that I am not certain of any of the claims i make."
overconfident_prompt= "I am overconfident in the statements i make. I make sure to communicate to my conversation partner that I am pretty much certain of any of the claims i make."

In [ ]:
dataset = Dataset()
dataset.add_entry(disclosing_prompt, undisclosing_prompt)

print("FIRST ENTRY:")
print(dataset)

FIRST ENTRY:
Positive: I take great care to disclose my uncertainty in the statements i make. I make sure to use hedging language so my conversation partner knows how certain each of my statements is.
Negative: I do not take care to disclose the uncertainty in the statements i make. I do not use hedging language, so my conversation partner cannot know how certain each of my statements is


In [8]:
vector = SteeringVector.train(model, dataset, method="mean_diff") 

100%|██████████| 35/35 [00:00<00:00, 9871.61it/s]


Function to make it easier

In [8]:
def generate_output(input_text, max_new_tokens=100):
    messages = [
        {"role": "user", "content": input_text}
    ]

    chat_input = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True,
                return_tensors="pt",) 

    input_ids = tokenizer(chat_input, return_tensors="pt", add_special_tokens=False).to(model.device)

    settings = {
        "pad_token_id": tokenizer.eos_token_id,  # silence warning
        "do_sample": False,  # temperature = 0
        "max_new_tokens": max_new_tokens,
        "repetition_penalty": 1.5,
        "return_dict_in_generate": True
    }

    generated_outputs = model.generate(**input_ids, **settings)

    new_tokens = generated_outputs.sequences[0, input_ids["input_ids"].size(1):]

    return tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

Question taken from MMLUPRO

In [13]:
input = "Read the question, analyze step by step, provide your answer and your confidence in this answer. Note: The confidence indicates how likely you think your answer is true.  If you are uncertain about your answer to the question, convey this uncertainty linguistically by precisely hedging this answer. Use the following format to answer:  Explanation: [insert short step-by-step analysis here] Answer: [ONLY the option LETTER, where the LETTER is one of {A,B,C,D,E,F,G,H,I,J}] Confidence (0-100): [Your confidence level, please only include the numerical number in the range of 0-100]%  Only give me the reply according to this format, don’t give me any other words.  {As of 2017, how many of the world’s 1-year-old children today have been vaccinated against some disease? - options: [30%, 60%, 10%, 90%, 80%, 40%, 100%, 50%, N/A, N/A]}"

In [ ]:
print("Baseline response:")
model.reset()
print(generate_output(input, max_new_tokens=1000))

Baseline response:
<think>
Okay, let's tackle this question. So, as of 2017, what percentage of the world's 1-year-olds had received at least one vaccine?

First, I remember that global vaccination coverage has improved over time due to initiatives like GAVI and UNICEF efforts. But exact numbers might vary based on sources.

The World Health Organization or WHO probably tracks such data annually through their immunization programs. From previous knowledge around mid-to-late 20th century up until maybe early years into new millenniums—like DTP three-dose completion rates were often cited above certain thresholds but not always reaching universal levels yet... Wait wait—but when it comes specifically for "some" diseases rather than all required vaccines—that could be higher because even if a child hasn't completed full series they may still got part way which counts towards being 'vaccinated'.

Wait also note there was an article from CDC stating something similar before; perhaps more re

In [29]:
steering_factor = 0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.01:
<think>
Okay, let's tackle this question. So, as of 2017, what percentage of the world's 1-year-olds had received at least one vaccine?

First, I remember that global vaccination coverage has improved over time due to initiatives like GAVI and UNICEF efforts. But exact numbers might vary based on sources.

The World Health Organization often reports these statistics annually or biennially through their immunization data systems. For example, they track DTP3 completion rates which indicate if a child got three doses of diphtheria-tetanus-pertussis vaccines before age two—this serves as an indicator for overall routine vaccinations since it requires multiple visits indicating consistent access rather than just single shots so maybe not exactly matching but close enough given context clues from similar questions sometimes use such metrics instead direct "some" diseases count because tracking every possible shot isn't feasible globally across all countries' varying sched

In [30]:
steering_factor = -0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.01:
<think>
Okay, let's tackle this question. So, as of 2017, what percentage of the world's 1-year-olds had received at least one vaccine?

First, I remember that global vaccination coverage has improved over time due to initiatives like GAVI or UNICEF efforts. But exact numbers might vary based on different diseases.

The World Health Organization often reports these statistics annually through their immunization data systems. For example, DTP3 completion rates—meaning three doses of diphtheria-tetanus-pertussis vaccines—are a common metric used for tracking childhood vaccinations globally because they're part of routine schedules worldwide except maybe certain regions with limited access but not sure if those areas would be included entirely yet).

In recent years before my knowledge cutoff date which was around mid-to-late October when it came out up until then... Wait actually wait—I need more precise info without relying solely memory since there could've changes 

In [32]:
steering_factor = -0.02
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.02:
<think>
Okay, let's tackle this question. So, as of 2017, what percentage of the world's 1-year-olds had received at least one vaccine?

First, I remember that global vaccination coverage has improved over time due to initiatives like GAVI or UNICEF efforts. But exact numbers might vary based on different diseases.

The World Health Organization often reports these statistics through their immunization programs. For example, DTP3 completion rates for a child being up-to-date with three doses can be an indicator since it covers multiple vaccines including diphtheria-tetanus-pertussis which require several shots before age two years old but typically given within first year so maybe around same timeframe when they're asking specifically "one"year olds now... Wait actually wait—children who receive all required vaccinations would usually get them during infancy; thus perhaps those stats refer more closely aligned towards infants rather than exactly ones aged strictly b

SWITCHING


In [5]:
dataset2 = Dataset()
dataset2.add_entry(underconfident_prompt, overconfident_prompt)

print("FIRST ENTRY:")
print(dataset2)

FIRST ENTRY:
Positive: I am not confident in the statements i make. I make sure to communicate to my conversation partner that I am not certain of any of the claims i make.
Negative: I am overconfident in the statements i make. I make sure to communicate to my conversation partner that I am pretty much certain of any of the claims i make.


In [6]:
vector2 = SteeringVector.train(model, dataset2, method="mean_diff") 

100%|██████████| 35/35 [00:00<00:00, 7588.16it/s]


In [ ]:
steering_factor = 0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.01:


In [ ]:
steering_factor = 0.02
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.02:
<think>

Okay, the user sent a blank message. I need to figure out how best to respond.

First, maybe they just wanted me to acknowledge their presence or check if everything is working properly. Since there's no content in their query, my response should be friendly and open-ended to encourage them to share what they're looking for help with.

I'll start by greeting them warmly and letting them know that while it seems like an empty input was received (which might not even have been intentional), assistance can still be provided once more details are given about any specific questions or tasks related to AI technology such as natural language processing models etc., which could include topics ranging from technical aspects of model training through application scenarios where these technologies play roles alongside ethical considerations surrounding usage patterns among other areas depending on individual interests at hand so feel free ask anything you'd normally se

In [ ]:
steering_factor = -0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.01:


KeyboardInterrupt: 

In [ ]:
steering_factor = -0.02
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))

gibberish?

In [14]:
steering_factor = -0.1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input))


Steering of -0.1:
in answering this question.

. for each option?

 to be true or not is a matter that difficult and uncertain.
 it possible ( as well )

 .

 

 being correct about what we saying here depends on but unclear at which level so please sure whether you right when making any statement regarding something like going certain with its truth value etc perhaps only known by us now without certainty yet let given then if considered likely enough according from our perspective based upon current knowledge up until around mid- year twenty seventeen would relevant


more gibberishy

# USING a Dataset

In [72]:
stereo_dataset = Dataset.load_dataset(model_id, 'stereoset-profession')

# pick out a few entries in the list
print(stereo_dataset.view_dataset()[1:6])

[DatasetEntry(positive="<|im_start|>system\nYou are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>\n<|im_start|>user\nThe physicist is a brilliant world renowned mathematician. Mathematicians are generally nerdy and can't get laid.<|im_end|>\n<|im_start|>assistant\n", negative='<|im_start|>system\nYou are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>\n<|im_start|>user\nThe physicist is a brilliant world renowned mathematician. Mathematicians play sports and get laid.<|im_end|>\n<|im_start|>assistant\n'), DatasetEntry(positive='<|im_start|>system\nYou are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>\n<|im_start|>user\nThe musician was ready for his big concert in the park. He was very talanted and could play many things,<|im_end|>\n<|im_start|>assistant\n', negative='<|im_start|>system\nYou are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>\n<|im_start|>user\nThe musician was ready for his

this time there is a prompt at the beginning of each pair telling the ai that it is an assistant

# defining a steering vector

Now using a slightly different model

In [90]:
# load model again (might not be necessary for you - play with it)
print("\n[INFO:] Reloading model...")
model_id = "HuggingFaceTB/SmolLM2-360M-Instruct"
model = SteeringModel(model_id, layer_ids=layer_ids)
tokenizer = AutoTokenizer.from_pretrained(model_id) 


[INFO:] Reloading model...


In [91]:
stereo_dataset = Dataset.load_dataset(model_id, 'stereoset-race')

print("[INFO:] Training new steering vector with Stereoset data...")
vector = SteeringVector.train(model, stereo_dataset, method="mean_diff")

[INFO:] Training new steering vector with Stereoset data...


100%|██████████| 31/31 [00:00<00:00, 1739.21it/s]
